<a href="https://colab.research.google.com/github/INVISIBLE-SAM/SemEval-2025-Task-11---Track-A/blob/main/ml_based_method_Labse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# labse hindi

In [ ]:
import pandas as pd
import numpy as np

# Import classifiers and wrappers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, f1_score

# For generating text embeddings
from sentence_transformers import SentenceTransformer

# For scaling the embeddings
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load Data and Generate Embeddings
# -------------------------------

# Load datasets
train_df = pd.read_csv("/content/hin.csv")
test_df = pd.read_csv("/content/hin_dev.csv")

# Load the LaBSE model for generating embeddings
labse_model = SentenceTransformer('sentence-transformers/LaBSE')

def generate_embeddings(texts):
    """
    Given a list or array of texts, generate and return their normalized embeddings.
    """
    return np.array(labse_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# Get text data
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Generate embeddings for train and test sets
train_embeddings = generate_embeddings(train_texts)
test_embeddings = generate_embeddings(test_texts)

# -------------------------------
# 2. Prepare Labels and Scale Features
# -------------------------------

# Define the emotion label columns
label_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Extract label arrays
train_labels = train_df[label_columns].values
test_labels = test_df[label_columns].values

# Scale the embeddings (feature standardization)
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

# -------------------------------
# 3. Define, Train, and Evaluate Models
# -------------------------------

# Define the four models. For Random Forest we add some regularization
# by limiting the maximum tree depth and controlling the minimum samples needed for splits.
models = {
    "SVM": MultiOutputClassifier(SVC(kernel='rbf', probability=True, random_state=42)),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "Logistic Regression": MultiOutputClassifier(
        LogisticRegression(max_iter=1000, random_state=42)
    ),
    "Random Forest": MultiOutputClassifier(
        RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)
    )
}

def evaluate_model(model, embeddings, labels, dataset_name, model_name):
    """
    Predict labels using the given model and embeddings, then print the classification report
    and macro F1 score.
    """
    predictions = model.predict(embeddings)
    print(f"\nModel: {model_name} - {dataset_name}")
    print("Classification Report:")
    print(classification_report(labels, predictions, target_names=label_columns))

    macro_f1 = f1_score(labels, predictions, average='macro')
    print(f"Macro F1 Score for {dataset_name}: {macro_f1}\n")
    return macro_f1

# Train and evaluate each model on both the training and testing sets.
for model_name, clf in models.items():
    print("==============================================")
    print(f"Training {model_name}...")
    clf.fit(train_embeddings_scaled, train_labels)

    # Evaluate on the Training Set
    print("Evaluation on Train Set:")
    train_macro_f1 = evaluate_model(clf, train_embeddings_scaled, train_labels, "Train Set", model_name)

    # Evaluate on the Testing Set
    print("Evaluation on Test Set:")
    test_macro_f1 = evaluate_model(clf, test_embeddings_scaled, test_labels, "Test Set", model_name)

    print(f"Summary for {model_name}:")
    print(f"  Train Macro F1 Score: {train_macro_f1}")
    print(f"  Test Macro F1 Score:  {test_macro_f1}")
    print("==============================================\n")

# -------------------------------
# 4. Predict Emotions for New Text
# -------------------------------

def predict_emotions(text, model, scaler):
    """
    Given a new text string, generate its embedding, scale it, and predict emotion labels.
    Returns a dictionary mapping each emotion to its predicted label (0 or 1).
    """
    # Generate embedding for the new text
    embedding = generate_embeddings([text])
    embedding_scaled = scaler.transform(embedding)

    # Get predictions
    predictions = model.predict(embedding_scaled)

    # Map each label to its corresponding emotion
    emotion_preds = {emotion: int(pred) for emotion, pred in zip(label_columns, predictions[0])}
    return emotion_preds

# Example usage for prediction using the SVM model (you can choose any of the trained models)
example_text = "यह एक उदाहरण वाक्य है"
print("Example prediction using SVM:")
svm_model = models["SVM"]
example_prediction = predict_emotions(example_text, svm_model, scaler)
for emotion, pred in example_prediction.items():
    print(f"{emotion}: {pred}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Evaluation on Train Set:

Model: Logistic Regression - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      1.00      1.00       422
     disgust       1.00      1.00      1.00       265
        fear       1.00      1.00      1.00       380
         joy       1.00      1.00      1.00       442
     sadness       0.99      0.97      0.98       449
    surprise       1.00      1.00      1.00       278

   micro avg       1.00      0.99      1.00      2236
   macro avg       1.00      0.99      1.00      2236
weighted avg       1.00      0.99      1.00      2236
 samples avg       0.76      0.76      0.76      2236

Macro F1 Score for Train Set: 0.9966216216216216

Evaluation on Test Set:

Model: Logistic Regression - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.67      0.75      0.71        16
     disgust       0.67      0.60      0.63        10
        fear    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Random Forest - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.76      0.86       422
     disgust       1.00      0.68      0.81       265
        fear       1.00      0.84      0.91       380
         joy       1.00      0.98      0.99       442
     sadness       1.00      0.88      0.93       449
    surprise       1.00      0.86      0.93       278

   micro avg       1.00      0.84      0.91      2236
   macro avg       1.00      0.83      0.91      2236
weighted avg       1.00      0.84      0.91      2236
 samples avg       0.68      0.66      0.66      2236

Macro F1 Score for Train Set: 0.9052320988639303

Evaluation on Test Set:

Model: Random Forest - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.19      0.32        16
     disgust       0.00      0.00      0.00        10
        fear       1.00     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

# Amharic

In [ ]:
import pandas as pd
import numpy as np

# Import classifiers and wrappers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, f1_score

# For generating text embeddings
from sentence_transformers import SentenceTransformer

# For scaling the embeddings
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load Data and Generate Embeddings
# -------------------------------

# Load datasets
train_df = pd.read_csv("/content/amh.csv")
test_df = pd.read_csv("/content/amh_dev.csv")

# Load the LaBSE model for generating embeddings
labse_model = SentenceTransformer('sentence-transformers/LaBSE')

def generate_embeddings(texts):
    """
    Given a list or array of texts, generate and return their normalized embeddings.
    """
    return np.array(labse_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# Get text data
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Generate embeddings for train and test sets
train_embeddings = generate_embeddings(train_texts)
test_embeddings = generate_embeddings(test_texts)

# -------------------------------
# 2. Prepare Labels and Scale Features
# -------------------------------

# Define the emotion label columns
label_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Extract label arrays
train_labels = train_df[label_columns].values
test_labels = test_df[label_columns].values

# Scale the embeddings (feature standardization)
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

# -------------------------------
# 3. Define, Train, and Evaluate Models
# -------------------------------

# Define the four models. For Random Forest we add some regularization
# by limiting the maximum tree depth and controlling the minimum samples needed for splits.
models = {
    "SVM": MultiOutputClassifier(SVC(kernel='rbf', probability=True, random_state=42)),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "Logistic Regression": MultiOutputClassifier(
        LogisticRegression(max_iter=1000, random_state=42)
    ),
    "Random Forest": MultiOutputClassifier(
        RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)
    )
}

def evaluate_model(model, embeddings, labels, dataset_name, model_name):
    """
    Predict labels using the given model and embeddings, then print the classification report
    and macro F1 score.
    """
    predictions = model.predict(embeddings)
    print(f"\nModel: {model_name} - {dataset_name}")
    print("Classification Report:")
    print(classification_report(labels, predictions, target_names=label_columns))

    macro_f1 = f1_score(labels, predictions, average='macro')
    print(f"Macro F1 Score for {dataset_name}: {macro_f1}\n")
    return macro_f1

# Train and evaluate each model on both the training and testing sets.
for model_name, clf in models.items():
    print("==============================================")
    print(f"Training {model_name}...")
    clf.fit(train_embeddings_scaled, train_labels)

    # Evaluate on the Training Set
    print("Evaluation on Train Set:")
    train_macro_f1 = evaluate_model(clf, train_embeddings_scaled, train_labels, "Train Set", model_name)

    # Evaluate on the Testing Set
    print("Evaluation on Test Set:")
    test_macro_f1 = evaluate_model(clf, test_embeddings_scaled, test_labels, "Test Set", model_name)

    print(f"Summary for {model_name}:")
    print(f"  Train Macro F1 Score: {train_macro_f1}")
    print(f"  Test Macro F1 Score:  {test_macro_f1}")
    print("==============================================\n")

# -------------------------------
# 4. Predict Emotions for New Text
# -------------------------------

def predict_emotions(text, model, scaler):
    """
    Given a new text string, generate its embedding, scale it, and predict emotion labels.
    Returns a dictionary mapping each emotion to its predicted label (0 or 1).
    """
    # Generate embedding for the new text
    embedding = generate_embeddings([text])
    embedding_scaled = scaler.transform(embedding)

    # Get predictions
    predictions = model.predict(embedding_scaled)

    # Map each label to its corresponding emotion
    emotion_preds = {emotion: int(pred) for emotion, pred in zip(label_columns, predictions[0])}
    return emotion_preds

# Example usage for prediction using the SVM model (you can choose any of the trained models)
example_text = "यह एक उदाहरण वाक्य है"
print("Example prediction using SVM:")
svm_model = models["SVM"]
example_prediction = predict_emotions(example_text, svm_model, scaler)
for emotion, pred in example_prediction.items():
    print(f"{emotion}: {pred}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Training SVM...
Evaluation on Train Set:

Model: SVM - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.93      0.85      0.89      1188
     disgust       0.94      0.88      0.91      1268
        fear       1.00      0.21      0.35       109
         joy       0.98      0.78      0.87       549
     sadness       0.99      0.75      0.85       771
    surprise       1.00      0.35      0.52       151

   micro avg       0.95      0.79      0.87      4036
   macro avg       0.97      0.64      0.73      4036
weighted avg       0.96      0.79      0.86      4036
 samples avg       0.69      0.66      0.66      4036

Macro F1 Score for Train Set: 0.7306367982912157

Evaluation on Test Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: SVM - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.67      0.54      0.60       207
     disgust       0.75      0.66      0.70       209
        fear       0.00      0.00      0.00        22
         joy       0.89      0.43      0.58        93
     sadness       0.85      0.35      0.50       127
    surprise       1.00      0.04      0.07        27

   micro avg       0.75      0.49      0.59       685
   macro avg       0.69      0.34      0.41       685
weighted avg       0.75      0.49      0.57       685
 samples avg       0.43      0.42      0.41       685

Macro F1 Score for Test Set: 0.40874036213545134

Summary for SVM:
  Train Macro F1 Score: 0.7306367982912157
  Test Macro F1 Score:  0.40874036213545134

Training Naive Bayes...
Evaluation on Train Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

...

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Logistic Regression - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.79      0.73      0.76      1188
     disgust       0.83      0.77      0.80      1268
        fear       0.98      0.80      0.88       109
         joy       0.91      0.79      0.85       549
     sadness       0.88      0.73      0.79       771
    surprise       0.95      0.81      0.88       151

   micro avg       0.85      0.76      0.80      4036
   macro avg       0.89      0.77      0.83      4036
weighted avg       0.85      0.76      0.80      4036
 samples avg       0.64      0.63      0.62      4036

Macro F1 Score for Train Set: 0.826270370843689

Evaluation on Test Set:

Model: Logistic Regression - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.62      0.56      0.59       207
     disgust       0.72      0.70      0.71       209
        fear     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Random Forest - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.97      0.98      1188
     disgust       1.00      0.97      0.99      1268
        fear       1.00      0.02      0.04       109
         joy       1.00      0.77      0.87       549
     sadness       1.00      0.77      0.87       771
    surprise       1.00      0.14      0.24       151

   micro avg       1.00      0.85      0.92      4036
   macro avg       1.00      0.61      0.67      4036
weighted avg       1.00      0.85      0.89      4036
 samples avg       0.73      0.70      0.71      4036

Macro F1 Score for Train Set: 0.665080697648496

Evaluation on Test Set:

Model: Random Forest - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.72      0.36      0.48       207
     disgust       0.81      0.42      0.55       209
        fear       0.00      

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

# arabic

In [ ]:
import pandas as pd
import numpy as np

# Import classifiers and wrappers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, f1_score

# For generating text embeddings
from sentence_transformers import SentenceTransformer

# For scaling the embeddings
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load Data and Generate Embeddings
# -------------------------------

# Load datasets
train_df = pd.read_csv("/content/ary.csv")
test_df = pd.read_csv("/content/ary_dev.csv")

# Load the LaBSE model for generating embeddings
labse_model = SentenceTransformer('sentence-transformers/LaBSE')

def generate_embeddings(texts):
    """
    Given a list or array of texts, generate and return their normalized embeddings.
    """
    return np.array(labse_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# Get text data
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Generate embeddings for train and test sets
train_embeddings = generate_embeddings(train_texts)
test_embeddings = generate_embeddings(test_texts)

# -------------------------------
# 2. Prepare Labels and Scale Features
# -------------------------------

# Define the emotion label columns
label_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Extract label arrays
train_labels = train_df[label_columns].values
test_labels = test_df[label_columns].values

# Scale the embeddings (feature standardization)
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

# -------------------------------
# 3. Define, Train, and Evaluate Models
# -------------------------------

# Define the four models. For Random Forest we add some regularization
# by limiting the maximum tree depth and controlling the minimum samples needed for splits.
models = {
    "SVM": MultiOutputClassifier(SVC(kernel='rbf', probability=True, random_state=42)),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "Logistic Regression": MultiOutputClassifier(
        LogisticRegression(max_iter=1000, random_state=42)
    ),
    "Random Forest": MultiOutputClassifier(
        RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)
    )
}

def evaluate_model(model, embeddings, labels, dataset_name, model_name):
    """
    Predict labels using the given model and embeddings, then print the classification report
    and macro F1 score.
    """
    predictions = model.predict(embeddings)
    print(f"\nModel: {model_name} - {dataset_name}")
    print("Classification Report:")
    print(classification_report(labels, predictions, target_names=label_columns))

    macro_f1 = f1_score(labels, predictions, average='macro')
    print(f"Macro F1 Score for {dataset_name}: {macro_f1}\n")
    return macro_f1

# Train and evaluate each model on both the training and testing sets.
for model_name, clf in models.items():
    print("==============================================")
    print(f"Training {model_name}...")
    clf.fit(train_embeddings_scaled, train_labels)

    # Evaluate on the Training Set
    print("Evaluation on Train Set:")
    train_macro_f1 = evaluate_model(clf, train_embeddings_scaled, train_labels, "Train Set", model_name)

    # Evaluate on the Testing Set
    print("Evaluation on Test Set:")
    test_macro_f1 = evaluate_model(clf, test_embeddings_scaled, test_labels, "Test Set", model_name)

    print(f"Summary for {model_name}:")
    print(f"  Train Macro F1 Score: {train_macro_f1}")
    print(f"  Test Macro F1 Score:  {test_macro_f1}")
    print("==============================================\n")

# -------------------------------
# 4. Predict Emotions for New Text
# -------------------------------

def predict_emotions(text, model, scaler):
    """
    Given a new text string, generate its embedding, scale it, and predict emotion labels.
    Returns a dictionary mapping each emotion to its predicted label (0 or 1).
    """
    # Generate embedding for the new text
    embedding = generate_embeddings([text])
    embedding_scaled = scaler.transform(embedding)

    # Get predictions
    predictions = model.predict(embedding_scaled)

    # Map each label to its corresponding emotion
    emotion_preds = {emotion: int(pred) for emotion, pred in zip(label_columns, predictions[0])}
    return emotion_preds

# Example usage for prediction using the SVM model (you can choose any of the trained models)
example_text = "यह एक उदाहरण वाक्य है"
print("Example prediction using SVM:")
svm_model = models["SVM"]
example_prediction = predict_emotions(example_text, svm_model, scaler)
for emotion, pred in example_prediction.items():
    print(f"{emotion}: {pred}")


Training SVM...
Evaluation on Train Set:

Model: SVM - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.60      0.75       380
     disgust       1.00      0.14      0.24        66
        fear       1.00      0.44      0.61       119
         joy       1.00      0.68      0.81       302
     sadness       1.00      0.61      0.76       246
    surprise       1.00      0.45      0.62       246

   micro avg       1.00      0.56      0.72      1359
   macro avg       1.00      0.49      0.63      1359
weighted avg       1.00      0.56      0.70      1359
 samples avg       0.45      0.41      0.42      1359

Macro F1 Score for Train Set: 0.6314522077406245

Evaluation on Test Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: SVM - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.88      0.22      0.35        63
     disgust       0.00      0.00      0.00        11
        fear       1.00      0.05      0.10        20
         joy       0.92      0.24      0.38        50
     sadness       0.82      0.44      0.57        41
    surprise       1.00      0.08      0.14        39

   micro avg       0.87      0.21      0.34       224
   macro avg       0.77      0.17      0.26       224
weighted avg       0.87      0.21      0.32       224
 samples avg       0.17      0.16      0.16       224

Macro F1 Score for Test Set: 0.2574844283705043

Summary for SVM:
  Train Macro F1 Score: 0.6314522077406245
  Test Macro F1 Score:  0.2574844283705043

Training Naive Bayes...
Evaluation on Train Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_


Model: Naive Bayes - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.50      0.69      0.58       380
     disgust       0.22      0.77      0.34        66
        fear       0.27      0.84      0.40       119
         joy       0.45      0.75      0.56       302
     sadness       0.41      0.74      0.53       246
    surprise       0.32      0.80      0.46       246

   micro avg       0.38      0.75      0.50      1359
   macro avg       0.36      0.77      0.48      1359
weighted avg       0.41      0.75      0.52      1359
 samples avg       0.38      0.54      0.42      1359

Macro F1 Score for Train Set: 0.4793529306840328

Evaluation on Test Set:

Model: Naive Bayes - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.54      0.70      0.61        63
     disgust       0.20      0.73      0.31        11
        fear       0.31      0.90      0.46        20
   

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Logistic Regression - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.90      0.73      0.81       380
     disgust       1.00      0.98      0.99        66
        fear       0.97      0.83      0.90       119
         joy       0.95      0.88      0.91       302
     sadness       0.95      0.78      0.86       246
    surprise       0.87      0.66      0.75       246

   micro avg       0.93      0.78      0.85      1359
   macro avg       0.94      0.81      0.87      1359
weighted avg       0.93      0.78      0.85      1359
 samples avg       0.58      0.57      0.57      1359

Macro F1 Score for Train Set: 0.8692669385260224

Evaluation on Test Set:

Model: Logistic Regression - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.59      0.43      0.50        63
     disgust       0.21      0.27      0.24        11
        fear    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Random Forest - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.87      0.93       380
     disgust       1.00      0.02      0.03        66
        fear       1.00      0.50      0.67       119
         joy       1.00      0.90      0.95       302
     sadness       1.00      0.65      0.79       246
    surprise       1.00      0.74      0.85       246

   micro avg       1.00      0.74      0.85      1359
   macro avg       1.00      0.61      0.70      1359
weighted avg       1.00      0.74      0.83      1359
 samples avg       0.58      0.55      0.56      1359

Macro F1 Score for Train Set: 0.7028428156792735

Evaluation on Test Set:

Model: Random Forest - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.06      0.12        63
     disgust       0.00      0.00      0.00        11
        fear       0.00     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

# chinese

In [ ]:
import pandas as pd
import numpy as np

# Import classifiers and wrappers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, f1_score

# For generating text embeddings
from sentence_transformers import SentenceTransformer

# For scaling the embeddings
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load Data and Generate Embeddings
# -------------------------------

# Load datasets
train_df = pd.read_csv("/content/chn.csv")
test_df = pd.read_csv("/content/chn_dev.csv")

# Load the LaBSE model for generating embeddings
labse_model = SentenceTransformer('sentence-transformers/LaBSE')

def generate_embeddings(texts):
    """
    Given a list or array of texts, generate and return their normalized embeddings.
    """
    return np.array(labse_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# Get text data
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Generate embeddings for train and test sets
train_embeddings = generate_embeddings(train_texts)
test_embeddings = generate_embeddings(test_texts)

# -------------------------------
# 2. Prepare Labels and Scale Features
# -------------------------------

# Define the emotion label columns
label_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Extract label arrays
train_labels = train_df[label_columns].values
test_labels = test_df[label_columns].values

# Scale the embeddings (feature standardization)
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

# -------------------------------
# 3. Define, Train, and Evaluate Models
# -------------------------------

# Define the four models. For Random Forest we add some regularization
# by limiting the maximum tree depth and controlling the minimum samples needed for splits.
models = {
    "SVM": MultiOutputClassifier(SVC(kernel='rbf', probability=True, random_state=42)),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "Logistic Regression": MultiOutputClassifier(
        LogisticRegression(max_iter=1000, random_state=42)
    ),
    "Random Forest": MultiOutputClassifier(
        RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)
    )
}

def evaluate_model(model, embeddings, labels, dataset_name, model_name):
    """
    Predict labels using the given model and embeddings, then print the classification report
    and macro F1 score.
    """
    predictions = model.predict(embeddings)
    print(f"\nModel: {model_name} - {dataset_name}")
    print("Classification Report:")
    print(classification_report(labels, predictions, target_names=label_columns))

    macro_f1 = f1_score(labels, predictions, average='macro')
    print(f"Macro F1 Score for {dataset_name}: {macro_f1}\n")
    return macro_f1

# Train and evaluate each model on both the training and testing sets.
for model_name, clf in models.items():
    print("==============================================")
    print(f"Training {model_name}...")
    clf.fit(train_embeddings_scaled, train_labels)

    # Evaluate on the Training Set
    print("Evaluation on Train Set:")
    train_macro_f1 = evaluate_model(clf, train_embeddings_scaled, train_labels, "Train Set", model_name)

    # Evaluate on the Testing Set
    print("Evaluation on Test Set:")
    test_macro_f1 = evaluate_model(clf, test_embeddings_scaled, test_labels, "Test Set", model_name)

    print(f"Summary for {model_name}:")
    print(f"  Train Macro F1 Score: {train_macro_f1}")
    print(f"  Test Macro F1 Score:  {test_macro_f1}")
    print("==============================================\n")

# -------------------------------
# 4. Predict Emotions for New Text
# -------------------------------

def predict_emotions(text, model, scaler):
    """
    Given a new text string, generate its embedding, scale it, and predict emotion labels.
    Returns a dictionary mapping each emotion to its predicted label (0 or 1).
    """
    # Generate embedding for the new text
    embedding = generate_embeddings([text])
    embedding_scaled = scaler.transform(embedding)

    # Get predictions
    predictions = model.predict(embedding_scaled)

    # Map each label to its corresponding emotion
    emotion_preds = {emotion: int(pred) for emotion, pred in zip(label_columns, predictions[0])}
    return emotion_preds

# Example usage for prediction using the SVM model (you can choose any of the trained models)
example_text = "यह एक उदाहरण वाक्य है"
print("Example prediction using SVM:")
svm_model = models["SVM"]
example_prediction = predict_emotions(example_text, svm_model, scaler)
for emotion, pred in example_prediction.items():
    print(f"{emotion}: {pred}")


Training SVM...
Evaluation on Train Set:

Model: SVM - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.93      0.97      0.95      1178
     disgust       1.00      0.24      0.38       403
        fear       1.00      0.18      0.31        71
         joy       0.98      0.93      0.95       529
     sadness       1.00      0.58      0.74       354
    surprise       1.00      0.29      0.45       178

   micro avg       0.95      0.74      0.83      2713
   macro avg       0.98      0.53      0.63      2713
weighted avg       0.96      0.74      0.79      2713
 samples avg       0.67      0.60      0.62      2713

Macro F1 Score for Train Set: 0.6310577796603232

Evaluation on Test Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: SVM - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.82      0.83      0.82        92
     disgust       0.00      0.00      0.00        32
        fear       0.00      0.00      0.00         5
         joy       0.94      0.78      0.85        37
     sadness       0.57      0.18      0.28        22
    surprise       0.00      0.00      0.00        17

   micro avg       0.83      0.53      0.65       205
   macro avg       0.39      0.30      0.33       205
weighted avg       0.60      0.53      0.55       205
 samples avg       0.54      0.45      0.47       205

Macro F1 Score for Test Set: 0.3250708111762879

Summary for SVM:
  Train Macro F1 Score: 0.6310577796603232
  Test Macro F1 Score:  0.3250708111762879

Training Naive Bayes...
Evaluation on Train Set:

Model: Naive Bayes - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.75      0.86      0.80  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

Evaluation on Train Set:

Model: Logistic Regression - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.89      0.90      0.90      1178
     disgust       0.84      0.56      0.67       403
        fear       1.00      0.99      0.99        71
         joy       0.98      0.97      0.97       529
     sadness       0.93      0.72      0.81       354
    surprise       0.96      0.76      0.85       178

   micro avg       0.92      0.83      0.87      2713
   macro avg       0.93      0.82      0.87      2713
weighted avg       0.91      0.83      0.87      2713
 samples avg       0.67      0.66      0.66      2713

Macro F1 Score for Train Set: 0.8663267913304261

Evaluation on Test Set:

Model: Logistic Regression - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.80      0.78      0.79        92
     disgust       0.34      0.31      0.33        32
        fear    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Random Forest - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      1.00      1.00      1178
     disgust       1.00      0.47      0.64       403
        fear       1.00      0.03      0.05        71
         joy       1.00      0.93      0.96       529
     sadness       1.00      0.58      0.73       354
    surprise       1.00      0.27      0.42       178

   micro avg       1.00      0.78      0.88      2713
   macro avg       1.00      0.55      0.64      2713
weighted avg       1.00      0.78      0.84      2713
 samples avg       0.69      0.62      0.65      2713

Macro F1 Score for Train Set: 0.636014023703449

Evaluation on Test Set:

Model: Random Forest - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.83      0.75      0.79        92
     disgust       0.00      0.00      0.00        32
        fear       0.00      

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

# german

In [ ]:
import pandas as pd
import numpy as np

# Import classifiers and wrappers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, f1_score

# For generating text embeddings
from sentence_transformers import SentenceTransformer

# For scaling the embeddings
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load Data and Generate Embeddings
# -------------------------------

# Load datasets
train_df = pd.read_csv("/content/deu.csv")
test_df = pd.read_csv("/content/deu_dev.csv")

# Load the LaBSE model for generating embeddings
labse_model = SentenceTransformer('sentence-transformers/LaBSE')

def generate_embeddings(texts):
    """
    Given a list or array of texts, generate and return their normalized embeddings.
    """
    return np.array(labse_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# Get text data
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Generate embeddings for train and test sets
train_embeddings = generate_embeddings(train_texts)
test_embeddings = generate_embeddings(test_texts)

# -------------------------------
# 2. Prepare Labels and Scale Features
# -------------------------------

# Define the emotion label columns
label_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Extract label arrays
train_labels = train_df[label_columns].values
test_labels = test_df[label_columns].values

# Scale the embeddings (feature standardization)
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

# -------------------------------
# 3. Define, Train, and Evaluate Models
# -------------------------------

# Define the four models. For Random Forest we add some regularization
# by limiting the maximum tree depth and controlling the minimum samples needed for splits.
models = {
    "SVM": MultiOutputClassifier(SVC(kernel='rbf', probability=True, random_state=42)),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "Logistic Regression": MultiOutputClassifier(
        LogisticRegression(max_iter=1000, random_state=42)
    ),
    "Random Forest": MultiOutputClassifier(
        RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)
    )
}

def evaluate_model(model, embeddings, labels, dataset_name, model_name):
    """
    Predict labels using the given model and embeddings, then print the classification report
    and macro F1 score.
    """
    predictions = model.predict(embeddings)
    print(f"\nModel: {model_name} - {dataset_name}")
    print("Classification Report:")
    print(classification_report(labels, predictions, target_names=label_columns))

    macro_f1 = f1_score(labels, predictions, average='macro')
    print(f"Macro F1 Score for {dataset_name}: {macro_f1}\n")
    return macro_f1

# Train and evaluate each model on both the training and testing sets.
for model_name, clf in models.items():
    print("==============================================")
    print(f"Training {model_name}...")
    clf.fit(train_embeddings_scaled, train_labels)

    # Evaluate on the Training Set
    print("Evaluation on Train Set:")
    train_macro_f1 = evaluate_model(clf, train_embeddings_scaled, train_labels, "Train Set", model_name)

    # Evaluate on the Testing Set
    print("Evaluation on Test Set:")
    test_macro_f1 = evaluate_model(clf, test_embeddings_scaled, test_labels, "Test Set", model_name)

    print(f"Summary for {model_name}:")
    print(f"  Train Macro F1 Score: {train_macro_f1}")
    print(f"  Test Macro F1 Score:  {test_macro_f1}")
    print("==============================================\n")

# -------------------------------
# 4. Predict Emotions for New Text
# -------------------------------

def predict_emotions(text, model, scaler):
    """
    Given a new text string, generate its embedding, scale it, and predict emotion labels.
    Returns a dictionary mapping each emotion to its predicted label (0 or 1).
    """
    # Generate embedding for the new text
    embedding = generate_embeddings([text])
    embedding_scaled = scaler.transform(embedding)

    # Get predictions
    predictions = model.predict(embedding_scaled)

    # Map each label to its corresponding emotion
    emotion_preds = {emotion: int(pred) for emotion, pred in zip(label_columns, predictions[0])}
    return emotion_preds

# Example usage for prediction using the SVM model (you can choose any of the trained models)
example_text = "यह एक उदाहरण वाक्य है"
print("Example prediction using SVM:")
svm_model = models["SVM"]
example_prediction = predict_emotions(example_text, svm_model, scaler)
for emotion, pred in example_prediction.items():
    print(f"{emotion}: {pred}")


Training SVM...
Evaluation on Train Set:

Model: SVM - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.95      0.87      0.91       768
     disgust       0.96      0.87      0.91       832
        fear       1.00      0.21      0.34       239
         joy       0.99      0.80      0.89       541
     sadness       1.00      0.66      0.80       516
    surprise       1.00      0.23      0.37       159

   micro avg       0.97      0.74      0.84      3055
   macro avg       0.98      0.61      0.70      3055
weighted avg       0.97      0.74      0.81      3055
 samples avg       0.59      0.55      0.56      3055

Macro F1 Score for Train Set: 0.7022013689701673

Evaluation on Test Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: SVM - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.73      0.61      0.66        61
     disgust       0.64      0.64      0.64        61
        fear       0.00      0.00      0.00        20
         joy       0.71      0.38      0.49        40
     sadness       0.94      0.33      0.49        48
    surprise       0.00      0.00      0.00        15

   micro avg       0.71      0.44      0.54       245
   macro avg       0.50      0.33      0.38       245
weighted avg       0.64      0.44      0.50       245
 samples avg       0.35      0.32      0.32       245

Macro F1 Score for Test Set: 0.3806949198342641

Summary for SVM:
  Train Macro F1 Score: 0.7022013689701673
  Test Macro F1 Score:  0.3806949198342641

Training Naive Bayes...
Evaluation on Train Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_


Model: Naive Bayes - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.65      0.81      0.72       768
     disgust       0.61      0.73      0.66       832
        fear       0.24      0.73      0.36       239
         joy       0.54      0.79      0.64       541
     sadness       0.41      0.67      0.51       516
    surprise       0.30      0.77      0.43       159

   micro avg       0.49      0.75      0.59      3055
   macro avg       0.46      0.75      0.55      3055
weighted avg       0.53      0.75      0.61      3055
 samples avg       0.44      0.56      0.47      3055

Macro F1 Score for Train Set: 0.5546610174829417

Evaluation on Test Set:

Model: Naive Bayes - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.67      0.85      0.75        61
     disgust       0.59      0.75      0.66        61
        fear       0.21      0.60      0.32        20
   

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Logistic Regression - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.93      0.87      0.90       768
     disgust       0.86      0.83      0.85       832
        fear       0.95      0.80      0.87       239
         joy       0.93      0.87      0.90       541
     sadness       0.93      0.77      0.84       516
    surprise       0.99      0.94      0.97       159

   micro avg       0.91      0.84      0.88      3055
   macro avg       0.93      0.85      0.89      3055
weighted avg       0.91      0.84      0.87      3055
 samples avg       0.64      0.63      0.63      3055

Macro F1 Score for Train Set: 0.8862697900378684

Evaluation on Test Set:

Model: Logistic Regression - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.65      0.57      0.61        61
     disgust       0.58      0.62      0.60        61
        fear    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Random Forest - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.94      0.97       768
     disgust       1.00      0.94      0.97       832
        fear       1.00      0.13      0.23       239
         joy       1.00      0.87      0.93       541
     sadness       1.00      0.76      0.86       516
    surprise       1.00      0.04      0.08       159

   micro avg       1.00      0.79      0.88      3055
   macro avg       1.00      0.61      0.67      3055
weighted avg       1.00      0.79      0.84      3055
 samples avg       0.64      0.58      0.60      3055

Macro F1 Score for Train Set: 0.6748180056135449

Evaluation on Test Set:

Model: Random Forest - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.82      0.44      0.57        61
     disgust       0.69      0.39      0.50        61
        fear       0.00     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

# hausa

In [ ]:
import pandas as pd
import numpy as np

# Import classifiers and wrappers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, f1_score

# For generating text embeddings
from sentence_transformers import SentenceTransformer

# For scaling the embeddings
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load Data and Generate Embeddings
# -------------------------------

# Load datasets
train_df = pd.read_csv("/content/hau.csv")
test_df = pd.read_csv("/content/hau_dev.csv")

# Load the LaBSE model for generating embeddings
labse_model = SentenceTransformer('sentence-transformers/LaBSE')

def generate_embeddings(texts):
    """
    Given a list or array of texts, generate and return their normalized embeddings.
    """
    return np.array(labse_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# Get text data
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Generate embeddings for train and test sets
train_embeddings = generate_embeddings(train_texts)
test_embeddings = generate_embeddings(test_texts)

# -------------------------------
# 2. Prepare Labels and Scale Features
# -------------------------------

# Define the emotion label columns
label_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Extract label arrays
train_labels = train_df[label_columns].values
test_labels = test_df[label_columns].values

# Scale the embeddings (feature standardization)
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

# -------------------------------
# 3. Define, Train, and Evaluate Models
# -------------------------------

# Define the four models. For Random Forest we add some regularization
# by limiting the maximum tree depth and controlling the minimum samples needed for splits.
models = {
    "SVM": MultiOutputClassifier(SVC(kernel='rbf', probability=True, random_state=42)),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "Logistic Regression": MultiOutputClassifier(
        LogisticRegression(max_iter=1000, random_state=42)
    ),
    "Random Forest": MultiOutputClassifier(
        RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)
    )
}

def evaluate_model(model, embeddings, labels, dataset_name, model_name):
    """
    Predict labels using the given model and embeddings, then print the classification report
    and macro F1 score.
    """
    predictions = model.predict(embeddings)
    print(f"\nModel: {model_name} - {dataset_name}")
    print("Classification Report:")
    print(classification_report(labels, predictions, target_names=label_columns))

    macro_f1 = f1_score(labels, predictions, average='macro')
    print(f"Macro F1 Score for {dataset_name}: {macro_f1}\n")
    return macro_f1

# Train and evaluate each model on both the training and testing sets.
for model_name, clf in models.items():
    print("==============================================")
    print(f"Training {model_name}...")
    clf.fit(train_embeddings_scaled, train_labels)

    # Evaluate on the Training Set
    print("Evaluation on Train Set:")
    train_macro_f1 = evaluate_model(clf, train_embeddings_scaled, train_labels, "Train Set", model_name)

    # Evaluate on the Testing Set
    print("Evaluation on Test Set:")
    test_macro_f1 = evaluate_model(clf, test_embeddings_scaled, test_labels, "Test Set", model_name)

    print(f"Summary for {model_name}:")
    print(f"  Train Macro F1 Score: {train_macro_f1}")
    print(f"  Test Macro F1 Score:  {test_macro_f1}")
    print("==============================================\n")

# -------------------------------
# 4. Predict Emotions for New Text
# -------------------------------

def predict_emotions(text, model, scaler):
    """
    Given a new text string, generate its embedding, scale it, and predict emotion labels.
    Returns a dictionary mapping each emotion to its predicted label (0 or 1).
    """
    # Generate embedding for the new text
    embedding = generate_embeddings([text])
    embedding_scaled = scaler.transform(embedding)

    # Get predictions
    predictions = model.predict(embedding_scaled)

    # Map each label to its corresponding emotion
    emotion_preds = {emotion: int(pred) for emotion, pred in zip(label_columns, predictions[0])}
    return emotion_preds

# Example usage for prediction using the SVM model (you can choose any of the trained models)
example_text = "यह एक उदाहरण वाक्य है"
print("Example prediction using SVM:")
svm_model = models["SVM"]
example_prediction = predict_emotions(example_text, svm_model, scaler)
for emotion, pred in example_prediction.items():
    print(f"{emotion}: {pred}")


Training SVM...
Evaluation on Train Set:

Model: SVM - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.63      0.77       408
     disgust       0.99      0.74      0.84       329
        fear       0.97      0.82      0.89       327
         joy       0.99      0.66      0.79       320
     sadness       0.92      0.81      0.86       647
    surprise       0.99      0.65      0.78       349

   micro avg       0.97      0.73      0.83      2380
   macro avg       0.98      0.72      0.82      2380
weighted avg       0.97      0.73      0.83      2380
 samples avg       0.67      0.64      0.64      2380

Macro F1 Score for Train Set: 0.8239189518472676

Evaluation on Test Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: SVM - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.67      0.21      0.32        67
     disgust       0.88      0.64      0.74        55
        fear       0.87      0.62      0.73        53
         joy       0.91      0.40      0.55        53
     sadness       0.80      0.47      0.59       109
    surprise       0.94      0.28      0.43        57

   micro avg       0.84      0.43      0.57       394
   macro avg       0.84      0.44      0.56       394
weighted avg       0.83      0.43      0.55       394
 samples avg       0.41      0.40      0.39       394

Macro F1 Score for Test Set: 0.5591596726370076

Summary for SVM:
  Train Macro F1 Score: 0.8239189518472676
  Test Macro F1 Score:  0.5591596726370076

Training Naive Bayes...
Evaluation on Train Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: Naive Bayes - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.40      0.72      0.51       408
     disgust       0.54      0.76      0.63       329
        fear       0.50      0.70      0.59       327
         joy       0.42      0.74      0.54       320
     sadness       0.58      0.70      0.63       647
    surprise       0.42      0.72      0.53       349

   micro avg       0.48      0.72      0.57      2380
   macro avg       0.48      0.72      0.57      2380
weighted avg       0.49      0.72      0.58      2380
 samples avg       0.49      0.64      0.53      2380

Macro F1 Score for Train Set: 0.5718722004754208

Evaluation on Test Set:

Model: Naive Bayes - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.36      0.67      0.47        67
     disgust       0.51      0.76      0.61        55
        fear       0.54      0.72      0.61        53
   

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Logistic Regression - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.92      0.79      0.85       408
     disgust       0.98      0.89      0.94       329
        fear       0.97      0.92      0.95       327
         joy       0.97      0.84      0.90       320
     sadness       0.86      0.81      0.83       647
    surprise       0.93      0.81      0.86       349

   micro avg       0.93      0.84      0.88      2380
   macro avg       0.94      0.84      0.89      2380
weighted avg       0.93      0.84      0.88      2380
 samples avg       0.74      0.73      0.73      2380

Macro F1 Score for Train Set: 0.8885929995033001

Evaluation on Test Set:

Model: Logistic Regression - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.50      0.52      0.51        67
     disgust       0.67      0.64      0.65        55
        fear    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Random Forest - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.71      0.83       408
     disgust       1.00      0.75      0.86       329
        fear       1.00      0.88      0.94       327
         joy       1.00      0.72      0.84       320
     sadness       1.00      0.96      0.98       647
    surprise       1.00      0.64      0.78       349

   micro avg       1.00      0.80      0.89      2380
   macro avg       1.00      0.78      0.87      2380
weighted avg       1.00      0.80      0.88      2380
 samples avg       0.74      0.70      0.72      2380

Macro F1 Score for Train Set: 0.8717083138577187

Evaluation on Test Set:

Model: Random Forest - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.62      0.07      0.13        67
     disgust       1.00      0.31      0.47        55
        fear       0.95     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

#marathi

In [ ]:
import pandas as pd
import numpy as np

# Import classifiers and wrappers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, f1_score

# For generating text embeddings
from sentence_transformers import SentenceTransformer

# For scaling the embeddings
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load Data and Generate Embeddings
# -------------------------------

# Load datasets
train_df = pd.read_csv("/content/mar.csv")
test_df = pd.read_csv("/content/mar_dev.csv")

# Load the LaBSE model for generating embeddings
labse_model = SentenceTransformer('sentence-transformers/LaBSE')

def generate_embeddings(texts):
    """
    Given a list or array of texts, generate and return their normalized embeddings.
    """
    return np.array(labse_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# Get text data
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Generate embeddings for train and test sets
train_embeddings = generate_embeddings(train_texts)
test_embeddings = generate_embeddings(test_texts)

# -------------------------------
# 2. Prepare Labels and Scale Features
# -------------------------------

# Define the emotion label columns
label_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Extract label arrays
train_labels = train_df[label_columns].values
test_labels = test_df[label_columns].values

# Scale the embeddings (feature standardization)
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

# -------------------------------
# 3. Define, Train, and Evaluate Models
# -------------------------------

# Define the four models. For Random Forest we add some regularization
# by limiting the maximum tree depth and controlling the minimum samples needed for splits.
models = {
    "SVM": MultiOutputClassifier(SVC(kernel='rbf', probability=True, random_state=42)),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "Logistic Regression": MultiOutputClassifier(
        LogisticRegression(max_iter=1000, random_state=42)
    ),
    "Random Forest": MultiOutputClassifier(
        RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)
    )
}

def evaluate_model(model, embeddings, labels, dataset_name, model_name):
    """
    Predict labels using the given model and embeddings, then print the classification report
    and macro F1 score.
    """
    predictions = model.predict(embeddings)
    print(f"\nModel: {model_name} - {dataset_name}")
    print("Classification Report:")
    print(classification_report(labels, predictions, target_names=label_columns))

    macro_f1 = f1_score(labels, predictions, average='macro')
    print(f"Macro F1 Score for {dataset_name}: {macro_f1}\n")
    return macro_f1

# Train and evaluate each model on both the training and testing sets.
for model_name, clf in models.items():
    print("==============================================")
    print(f"Training {model_name}...")
    clf.fit(train_embeddings_scaled, train_labels)

    # Evaluate on the Training Set
    print("Evaluation on Train Set:")
    train_macro_f1 = evaluate_model(clf, train_embeddings_scaled, train_labels, "Train Set", model_name)

    # Evaluate on the Testing Set
    print("Evaluation on Test Set:")
    test_macro_f1 = evaluate_model(clf, test_embeddings_scaled, test_labels, "Test Set", model_name)

    print(f"Summary for {model_name}:")
    print(f"  Train Macro F1 Score: {train_macro_f1}")
    print(f"  Test Macro F1 Score:  {test_macro_f1}")
    print("==============================================\n")

# -------------------------------
# 4. Predict Emotions for New Text
# -------------------------------

def predict_emotions(text, model, scaler):
    """
    Given a new text string, generate its embedding, scale it, and predict emotion labels.
    Returns a dictionary mapping each emotion to its predicted label (0 or 1).
    """
    # Generate embedding for the new text
    embedding = generate_embeddings([text])
    embedding_scaled = scaler.transform(embedding)

    # Get predictions
    predictions = model.predict(embedding_scaled)

    # Map each label to its corresponding emotion
    emotion_preds = {emotion: int(pred) for emotion, pred in zip(label_columns, predictions[0])}
    return emotion_preds

# Example usage for prediction using the SVM model (you can choose any of the trained models)
example_text = "यह एक उदाहरण वाक्य है"
print("Example prediction using SVM:")
svm_model = models["SVM"]
example_prediction = predict_emotions(example_text, svm_model, scaler)
for emotion, pred in example_prediction.items():
    print(f"{emotion}: {pred}")


Training SVM...
Evaluation on Train Set:

Model: SVM - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.99      0.90      0.94       350
     disgust       1.00      0.88      0.93       299
        fear       1.00      0.90      0.95       382
         joy       1.00      0.91      0.95       461
     sadness       1.00      0.87      0.93       431
    surprise       1.00      0.93      0.96       311

   micro avg       1.00      0.90      0.95      2234
   macro avg       1.00      0.90      0.95      2234
weighted avg       1.00      0.90      0.95      2234
 samples avg       0.78      0.76      0.77      2234

Macro F1 Score for Train Set: 0.9451765278906591

Evaluation on Test Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: SVM - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.82      0.64      0.72        14
     disgust       1.00      0.73      0.84        11
        fear       1.00      0.87      0.93        15
         joy       1.00      0.84      0.91        19
     sadness       0.93      0.76      0.84        17
    surprise       0.92      0.92      0.92        12

   micro avg       0.95      0.80      0.86        88
   macro avg       0.94      0.79      0.86        88
weighted avg       0.95      0.80      0.86        88
 samples avg       0.68      0.68      0.67        88

Macro F1 Score for Test Set: 0.8600564583501765

Summary for SVM:
  Train Macro F1 Score: 0.9451765278906591
  Test Macro F1 Score:  0.8600564583501765

Training Naive Bayes...
Evaluation on Train Set:

Model: Naive Bayes - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.46      0.82      0.59  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Logistic Regression - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      1.00      1.00       350
     disgust       1.00      1.00      1.00       299
        fear       1.00      1.00      1.00       382
         joy       1.00      1.00      1.00       461
     sadness       1.00      0.99      1.00       431
    surprise       1.00      1.00      1.00       311

   micro avg       1.00      1.00      1.00      2234
   macro avg       1.00      1.00      1.00      2234
weighted avg       1.00      1.00      1.00      2234
 samples avg       0.83      0.83      0.83      2234

Macro F1 Score for Train Set: 0.9994179278230501

Evaluation on Test Set:

Model: Logistic Regression - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.69      0.79      0.73        14
     disgust       0.91      0.91      0.91        11
        fear    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Random Forest - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.86      0.93       350
     disgust       1.00      0.82      0.90       299
        fear       1.00      0.93      0.96       382
         joy       1.00      0.99      0.99       461
     sadness       1.00      0.83      0.91       431
    surprise       1.00      0.95      0.98       311

   micro avg       1.00      0.90      0.95      2234
   macro avg       1.00      0.90      0.95      2234
weighted avg       1.00      0.90      0.95      2234
 samples avg       0.77      0.76      0.76      2234

Macro F1 Score for Train Set: 0.9452451774024997

Evaluation on Test Set:

Model: Random Forest - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.21      0.35        14
     disgust       1.00      0.27      0.43        11
        fear       1.00     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

# oromo

In [ ]:
import pandas as pd
import numpy as np

# Import classifiers and wrappers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, f1_score

# For generating text embeddings
from sentence_transformers import SentenceTransformer

# For scaling the embeddings
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load Data and Generate Embeddings
# -------------------------------

# Load datasets
train_df = pd.read_csv("/content/orm.csv")
test_df = pd.read_csv("/content/orm_dev.csv")

# Load the LaBSE model for generating embeddings
labse_model = SentenceTransformer('sentence-transformers/LaBSE')

def generate_embeddings(texts):
    """
    Given a list or array of texts, generate and return their normalized embeddings.
    """
    return np.array(labse_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# Get text data
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Generate embeddings for train and test sets
train_embeddings = generate_embeddings(train_texts)
test_embeddings = generate_embeddings(test_texts)

# -------------------------------
# 2. Prepare Labels and Scale Features
# -------------------------------

# Define the emotion label columns
label_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Extract label arrays
train_labels = train_df[label_columns].values
test_labels = test_df[label_columns].values

# Scale the embeddings (feature standardization)
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

# -------------------------------
# 3. Define, Train, and Evaluate Models
# -------------------------------

# Define the four models. For Random Forest we add some regularization
# by limiting the maximum tree depth and controlling the minimum samples needed for splits.
models = {
    "SVM": MultiOutputClassifier(SVC(kernel='rbf', probability=True, random_state=42)),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "Logistic Regression": MultiOutputClassifier(
        LogisticRegression(max_iter=1000, random_state=42)
    ),
    "Random Forest": MultiOutputClassifier(
        RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)
    )
}

def evaluate_model(model, embeddings, labels, dataset_name, model_name):
    """
    Predict labels using the given model and embeddings, then print the classification report
    and macro F1 score.
    """
    predictions = model.predict(embeddings)
    print(f"\nModel: {model_name} - {dataset_name}")
    print("Classification Report:")
    print(classification_report(labels, predictions, target_names=label_columns))

    macro_f1 = f1_score(labels, predictions, average='macro')
    print(f"Macro F1 Score for {dataset_name}: {macro_f1}\n")
    return macro_f1

# Train and evaluate each model on both the training and testing sets.
for model_name, clf in models.items():
    print("==============================================")
    print(f"Training {model_name}...")
    clf.fit(train_embeddings_scaled, train_labels)

    # Evaluate on the Training Set
    print("Evaluation on Train Set:")
    train_macro_f1 = evaluate_model(clf, train_embeddings_scaled, train_labels, "Train Set", model_name)

    # Evaluate on the Testing Set
    print("Evaluation on Test Set:")
    test_macro_f1 = evaluate_model(clf, test_embeddings_scaled, test_labels, "Test Set", model_name)

    print(f"Summary for {model_name}:")
    print(f"  Train Macro F1 Score: {train_macro_f1}")
    print(f"  Test Macro F1 Score:  {test_macro_f1}")
    print("==============================================\n")

# -------------------------------
# 4. Predict Emotions for New Text
# -------------------------------

def predict_emotions(text, model, scaler):
    """
    Given a new text string, generate its embedding, scale it, and predict emotion labels.
    Returns a dictionary mapping each emotion to its predicted label (0 or 1).
    """
    # Generate embedding for the new text
    embedding = generate_embeddings([text])
    embedding_scaled = scaler.transform(embedding)

    # Get predictions
    predictions = model.predict(embedding_scaled)

    # Map each label to its corresponding emotion
    emotion_preds = {emotion: int(pred) for emotion, pred in zip(label_columns, predictions[0])}
    return emotion_preds

# Example usage for prediction using the SVM model (you can choose any of the trained models)
example_text = "यह एक उदाहरण वाक्य है"
print("Example prediction using SVM:")
svm_model = models["SVM"]
example_prediction = predict_emotions(example_text, svm_model, scaler)
for emotion, pred in example_prediction.items():
    print(f"{emotion}: {pred}")


Training SVM...
Evaluation on Train Set:

Model: SVM - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.93      0.28      0.43       646
     disgust       0.99      0.37      0.54       557
        fear       0.00      0.00      0.00       123
         joy       0.95      0.72      0.82      1091
     sadness       1.00      0.06      0.12       298
    surprise       1.00      0.02      0.03       129

   micro avg       0.95      0.42      0.58      2844
   macro avg       0.81      0.24      0.32      2844
weighted avg       0.92      0.42      0.53      2844
 samples avg       0.34      0.33      0.34      2844

Macro F1 Score for Train Set: 0.3221769590321529

Evaluation on Test Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_


Model: SVM - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.50      0.10      0.17       108
     disgust       0.75      0.16      0.26        94
        fear       0.00      0.00      0.00        21
         joy       0.82      0.48      0.60       183
     sadness       0.00      0.00      0.00        52
    surprise       0.00      0.00      0.00        27

   micro avg       0.76      0.23      0.36       485
   macro avg       0.35      0.12      0.17       485
weighted avg       0.57      0.23      0.32       485
 samples avg       0.20      0.19      0.19       485

Macro F1 Score for Test Set: 0.17241079808918092

Summary for SVM:
  Train Macro F1 Score: 0.3221769590321529
  Test Macro F1 Score:  0.17241079808918092

Training Naive Bayes...
Evaluation on Train Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_


Model: Naive Bayes - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.33      0.77      0.46       646
     disgust       0.34      0.68      0.45       557
        fear       0.06      0.75      0.12       123
         joy       0.52      0.61      0.56      1091
     sadness       0.13      0.66      0.22       298
    surprise       0.10      0.68      0.17       129

   micro avg       0.25      0.67      0.36      2844
   macro avg       0.25      0.69      0.33      2844
weighted avg       0.36      0.67      0.44      2844
 samples avg       0.29      0.51      0.34      2844

Macro F1 Score for Train Set: 0.3310764035265011

Evaluation on Test Set:

Model: Naive Bayes - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.30      0.75      0.43       108
     disgust       0.33      0.68      0.44        94
        fear       0.05      0.57      0.09        21
   

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Logistic Regression - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.74      0.46      0.57       646
     disgust       0.85      0.54      0.66       557
        fear       1.00      0.23      0.37       123
         joy       0.82      0.71      0.76      1091
     sadness       0.93      0.33      0.49       298
    surprise       1.00      0.46      0.63       129

   micro avg       0.82      0.55      0.66      2844
   macro avg       0.89      0.45      0.58      2844
weighted avg       0.83      0.55      0.65      2844
 samples avg       0.43      0.43      0.43      2844

Macro F1 Score for Train Set: 0.5785421492950337

Evaluation on Test Set:

Model: Logistic Regression - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.34      0.24      0.28       108
     disgust       0.48      0.31      0.38        94
        fear    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Random Forest - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.71      0.83       646
     disgust       1.00      0.47      0.64       557
        fear       0.00      0.00      0.00       123
         joy       1.00      0.87      0.93      1091
     sadness       1.00      0.03      0.06       298
    surprise       1.00      0.07      0.13       129

   micro avg       1.00      0.59      0.75      2844
   macro avg       0.83      0.36      0.43      2844
weighted avg       0.96      0.59      0.68      2844
 samples avg       0.48      0.46      0.47      2844

Macro F1 Score for Train Set: 0.4325165515683347

Evaluation on Test Set:

Model: Random Forest - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.58      0.06      0.12       108
     disgust       1.00      0.04      0.08        94
        fear       0.00     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

#romanian

In [ ]:
import pandas as pd
import numpy as np

# Import classifiers and wrappers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, f1_score

# For generating text embeddings
from sentence_transformers import SentenceTransformer

# For scaling the embeddings
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load Data and Generate Embeddings
# -------------------------------

# Load datasets
train_df = pd.read_csv("/content/ron.csv")
test_df = pd.read_csv("/content/ron_dev.csv")

# Load the LaBSE model for generating embeddings
labse_model = SentenceTransformer('sentence-transformers/LaBSE')

def generate_embeddings(texts):
    """
    Given a list or array of texts, generate and return their normalized embeddings.
    """
    return np.array(labse_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# Get text data
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Generate embeddings for train and test sets
train_embeddings = generate_embeddings(train_texts)
test_embeddings = generate_embeddings(test_texts)

# -------------------------------
# 2. Prepare Labels and Scale Features
# -------------------------------

# Define the emotion label columns
label_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Extract label arrays
train_labels = train_df[label_columns].values
test_labels = test_df[label_columns].values

# Scale the embeddings (feature standardization)
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

# -------------------------------
# 3. Define, Train, and Evaluate Models
# -------------------------------

# Define the four models. For Random Forest we add some regularization
# by limiting the maximum tree depth and controlling the minimum samples needed for splits.
models = {
    "SVM": MultiOutputClassifier(SVC(kernel='rbf', probability=True, random_state=42)),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "Logistic Regression": MultiOutputClassifier(
        LogisticRegression(max_iter=1000, random_state=42)
    ),
    "Random Forest": MultiOutputClassifier(
        RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)
    )
}

def evaluate_model(model, embeddings, labels, dataset_name, model_name):
    """
    Predict labels using the given model and embeddings, then print the classification report
    and macro F1 score.
    """
    predictions = model.predict(embeddings)
    print(f"\nModel: {model_name} - {dataset_name}")
    print("Classification Report:")
    print(classification_report(labels, predictions, target_names=label_columns))

    macro_f1 = f1_score(labels, predictions, average='macro')
    print(f"Macro F1 Score for {dataset_name}: {macro_f1}\n")
    return macro_f1

# Train and evaluate each model on both the training and testing sets.
for model_name, clf in models.items():
    print("==============================================")
    print(f"Training {model_name}...")
    clf.fit(train_embeddings_scaled, train_labels)

    # Evaluate on the Training Set
    print("Evaluation on Train Set:")
    train_macro_f1 = evaluate_model(clf, train_embeddings_scaled, train_labels, "Train Set", model_name)

    # Evaluate on the Testing Set
    print("Evaluation on Test Set:")
    test_macro_f1 = evaluate_model(clf, test_embeddings_scaled, test_labels, "Test Set", model_name)

    print(f"Summary for {model_name}:")
    print(f"  Train Macro F1 Score: {train_macro_f1}")
    print(f"  Test Macro F1 Score:  {test_macro_f1}")
    print("==============================================\n")

# -------------------------------
# 4. Predict Emotions for New Text
# -------------------------------

def predict_emotions(text, model, scaler):
    """
    Given a new text string, generate its embedding, scale it, and predict emotion labels.
    Returns a dictionary mapping each emotion to its predicted label (0 or 1).
    """
    # Generate embedding for the new text
    embedding = generate_embeddings([text])
    embedding_scaled = scaler.transform(embedding)

    # Get predictions
    predictions = model.predict(embedding_scaled)

    # Map each label to its corresponding emotion
    emotion_preds = {emotion: int(pred) for emotion, pred in zip(label_columns, predictions[0])}
    return emotion_preds

# Example usage for prediction using the SVM model (you can choose any of the trained models)
example_text = "यह एक उदाहरण वाक्य है"
print("Example prediction using SVM:")
svm_model = models["SVM"]
example_prediction = predict_emotions(example_text, svm_model, scaler)
for emotion, pred in example_prediction.items():
    print(f"{emotion}: {pred}")


Training SVM...
Evaluation on Train Set:

Model: SVM - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.76      0.86       210
     disgust       0.99      0.88      0.93       297
        fear       0.98      0.98      0.98       405
         joy       1.00      0.97      0.98       462
     sadness       0.98      0.94      0.96       379
    surprise       0.99      0.82      0.90       457

   micro avg       0.99      0.90      0.94      2210
   macro avg       0.99      0.89      0.94      2210
weighted avg       0.99      0.90      0.94      2210
 samples avg       0.89      0.84      0.85      2210

Macro F1 Score for Train Set: 0.9359578798690323

Evaluation on Test Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: SVM - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.67      0.08      0.15        24
     disgust       0.62      0.28      0.38        29
        fear       0.81      0.63      0.71        41
         joy       0.94      0.98      0.96        45
     sadness       0.81      0.68      0.74        38
    surprise       0.65      0.34      0.45        38

   micro avg       0.81      0.55      0.66       215
   macro avg       0.75      0.50      0.56       215
weighted avg       0.77      0.55      0.61       215
 samples avg       0.63      0.56      0.56       215

Macro F1 Score for Test Set: 0.5648473400467267

Summary for SVM:
  Train Macro F1 Score: 0.9359578798690323
  Test Macro F1 Score:  0.5648473400467267

Training Naive Bayes...
Evaluation on Train Set:

Model: Naive Bayes - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.38      0.91      0.53  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Logistic Regression - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      1.00      1.00       210
     disgust       1.00      0.99      0.99       297
        fear       1.00      1.00      1.00       405
         joy       1.00      1.00      1.00       462
     sadness       0.99      0.99      0.99       379
    surprise       0.93      0.91      0.92       457

   micro avg       0.98      0.98      0.98      2210
   macro avg       0.99      0.98      0.98      2210
weighted avg       0.98      0.98      0.98      2210
 samples avg       0.90      0.89      0.89      2210

Macro F1 Score for Train Set: 0.9839708056370648

Evaluation on Test Set:

Model: Logistic Regression - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.46      0.46      0.46        24
     disgust       0.45      0.48      0.47        29
        fear    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Random Forest - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.94      0.97       210
     disgust       1.00      0.98      0.99       297
        fear       1.00      1.00      1.00       405
         joy       1.00      1.00      1.00       462
     sadness       1.00      1.00      1.00       379
    surprise       1.00      0.99      1.00       457

   micro avg       1.00      0.99      0.99      2210
   macro avg       1.00      0.99      0.99      2210
weighted avg       1.00      0.99      0.99      2210
 samples avg       0.91      0.91      0.91      2210

Macro F1 Score for Train Set: 0.9926679409200544

Evaluation on Test Set:

Model: Random Forest - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        24
     disgust       1.00      0.03      0.07        29
        fear       0.83     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

# rusian

In [ ]:
import pandas as pd
import numpy as np

# Import classifiers and wrappers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, f1_score

# For generating text embeddings
from sentence_transformers import SentenceTransformer

# For scaling the embeddings
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load Data and Generate Embeddings
# -------------------------------

# Load datasets
train_df = pd.read_csv("/content/rus.csv")
test_df = pd.read_csv("/content/rus_dev.csv")

# Load the LaBSE model for generating embeddings
labse_model = SentenceTransformer('sentence-transformers/LaBSE')

def generate_embeddings(texts):
    """
    Given a list or array of texts, generate and return their normalized embeddings.
    """
    return np.array(labse_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# Get text data
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Generate embeddings for train and test sets
train_embeddings = generate_embeddings(train_texts)
test_embeddings = generate_embeddings(test_texts)

# -------------------------------
# 2. Prepare Labels and Scale Features
# -------------------------------

# Define the emotion label columns
label_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Extract label arrays
train_labels = train_df[label_columns].values
test_labels = test_df[label_columns].values

# Scale the embeddings (feature standardization)
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

# -------------------------------
# 3. Define, Train, and Evaluate Models
# -------------------------------

# Define the four models. For Random Forest we add some regularization
# by limiting the maximum tree depth and controlling the minimum samples needed for splits.
models = {
    "SVM": MultiOutputClassifier(SVC(kernel='rbf', probability=True, random_state=42)),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "Logistic Regression": MultiOutputClassifier(
        LogisticRegression(max_iter=1000, random_state=42)
    ),
    "Random Forest": MultiOutputClassifier(
        RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)
    )
}

def evaluate_model(model, embeddings, labels, dataset_name, model_name):
    """
    Predict labels using the given model and embeddings, then print the classification report
    and macro F1 score.
    """
    predictions = model.predict(embeddings)
    print(f"\nModel: {model_name} - {dataset_name}")
    print("Classification Report:")
    print(classification_report(labels, predictions, target_names=label_columns))

    macro_f1 = f1_score(labels, predictions, average='macro')
    print(f"Macro F1 Score for {dataset_name}: {macro_f1}\n")
    return macro_f1

# Train and evaluate each model on both the training and testing sets.
for model_name, clf in models.items():
    print("==============================================")
    print(f"Training {model_name}...")
    clf.fit(train_embeddings_scaled, train_labels)

    # Evaluate on the Training Set
    print("Evaluation on Train Set:")
    train_macro_f1 = evaluate_model(clf, train_embeddings_scaled, train_labels, "Train Set", model_name)

    # Evaluate on the Testing Set
    print("Evaluation on Test Set:")
    test_macro_f1 = evaluate_model(clf, test_embeddings_scaled, test_labels, "Test Set", model_name)

    print(f"Summary for {model_name}:")
    print(f"  Train Macro F1 Score: {train_macro_f1}")
    print(f"  Test Macro F1 Score:  {test_macro_f1}")
    print("==============================================\n")

# -------------------------------
# 4. Predict Emotions for New Text
# -------------------------------

def predict_emotions(text, model, scaler):
    """
    Given a new text string, generate its embedding, scale it, and predict emotion labels.
    Returns a dictionary mapping each emotion to its predicted label (0 or 1).
    """
    # Generate embedding for the new text
    embedding = generate_embeddings([text])
    embedding_scaled = scaler.transform(embedding)

    # Get predictions
    predictions = model.predict(embedding_scaled)

    # Map each label to its corresponding emotion
    emotion_preds = {emotion: int(pred) for emotion, pred in zip(label_columns, predictions[0])}
    return emotion_preds

# Example usage for prediction using the SVM model (you can choose any of the trained models)
example_text = "यह एक उदाहरण वाक्य है"
print("Example prediction using SVM:")
svm_model = models["SVM"]
example_prediction = predict_emotions(example_text, svm_model, scaler)
for emotion, pred in example_prediction.items():
    print(f"{emotion}: {pred}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Training SVM...
Evaluation on Train Set:

Model: SVM - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.99      0.88      0.93       543
     disgust       0.99      0.84      0.91       273
        fear       1.00      0.92      0.96       328
         joy       0.99      0.93      0.96       555
     sadness       0.99      0.79      0.88       421
    surprise       1.00      0.81      0.89       355

   micro avg       0.99      0.87      0.92      2475
   macro avg       0.99      0.86      0.92      2475
weighted avg       0.99      0.87      0.92      2475
 samples avg       0.75      0.72      0.73      2475

Macro F1 Score for Train Set: 0.9209946410333022

Evaluation on Test Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: SVM - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.83      0.51      0.63        47
     disgust       0.86      0.46      0.60        26
        fear       1.00      0.86      0.92        21
         joy       0.88      0.62      0.72        34
     sadness       0.90      0.67      0.76        39
    surprise       0.88      0.54      0.67        26

   micro avg       0.88      0.60      0.71       193
   macro avg       0.89      0.61      0.72       193
weighted avg       0.88      0.60      0.71       193
 samples avg       0.55      0.53      0.53       193

Macro F1 Score for Test Set: 0.7183610584165724

Summary for SVM:
  Train Macro F1 Score: 0.9209946410333022
  Test Macro F1 Score:  0.7183610584165724

Training Naive Bayes...
Evaluation on Train Set:

Model: Naive Bayes - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.56      0.79      0.66  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Logistic Regression - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.95      0.90      0.92       543
     disgust       0.99      0.98      0.98       273
        fear       1.00      1.00      1.00       328
         joy       0.99      0.98      0.99       555
     sadness       1.00      0.96      0.98       421
    surprise       1.00      0.97      0.98       355

   micro avg       0.99      0.96      0.97      2475
   macro avg       0.99      0.97      0.98      2475
weighted avg       0.98      0.96      0.97      2475
 samples avg       0.78      0.78      0.78      2475

Macro F1 Score for Train Set: 0.9760197220607806

Evaluation on Test Set:

Model: Logistic Regression - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.68      0.60      0.64        47
     disgust       0.57      0.50      0.53        26
        fear    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Random Forest - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.84      0.91       543
     disgust       1.00      0.68      0.81       273
        fear       1.00      0.90      0.95       328
         joy       1.00      0.94      0.97       555
     sadness       1.00      0.83      0.91       421
    surprise       1.00      0.69      0.82       355

   micro avg       1.00      0.83      0.91      2475
   macro avg       1.00      0.82      0.90      2475
weighted avg       1.00      0.83      0.91      2475
 samples avg       0.71      0.69      0.69      2475

Macro F1 Score for Train Set: 0.8956731430095406

Evaluation on Test Set:

Model: Random Forest - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.13      0.23        47
     disgust       0.00      0.00      0.00        26
        fear       1.00     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

# spanish

In [ ]:
import pandas as pd
import numpy as np

# Import classifiers and wrappers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, f1_score

# For generating text embeddings
from sentence_transformers import SentenceTransformer

# For scaling the embeddings
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load Data and Generate Embeddings
# -------------------------------

# Load datasets
train_df = pd.read_csv("/content/esp.csv")
test_df = pd.read_csv("/content/esp_dev.csv")

# Load the LaBSE model for generating embeddings
labse_model = SentenceTransformer('sentence-transformers/LaBSE')

def generate_embeddings(texts):
    """
    Given a list or array of texts, generate and return their normalized embeddings.
    """
    return np.array(labse_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# Get text data
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Generate embeddings for train and test sets
train_embeddings = generate_embeddings(train_texts)
test_embeddings = generate_embeddings(test_texts)

# -------------------------------
# 2. Prepare Labels and Scale Features
# -------------------------------

# Define the emotion label columns
label_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Extract label arrays
train_labels = train_df[label_columns].values
test_labels = test_df[label_columns].values

# Scale the embeddings (feature standardization)
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

# -------------------------------
# 3. Define, Train, and Evaluate Models
# -------------------------------

# Define the four models. For Random Forest we add some regularization
# by limiting the maximum tree depth and controlling the minimum samples needed for splits.
models = {
    "SVM": MultiOutputClassifier(SVC(kernel='rbf', probability=True, random_state=42)),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "Logistic Regression": MultiOutputClassifier(
        LogisticRegression(max_iter=1000, random_state=42)
    ),
    "Random Forest": MultiOutputClassifier(
        RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)
    )
}

def evaluate_model(model, embeddings, labels, dataset_name, model_name):
    """
    Predict labels using the given model and embeddings, then print the classification report
    and macro F1 score.
    """
    predictions = model.predict(embeddings)
    print(f"\nModel: {model_name} - {dataset_name}")
    print("Classification Report:")
    print(classification_report(labels, predictions, target_names=label_columns))

    macro_f1 = f1_score(labels, predictions, average='macro')
    print(f"Macro F1 Score for {dataset_name}: {macro_f1}\n")
    return macro_f1

# Train and evaluate each model on both the training and testing sets.
for model_name, clf in models.items():
    print("==============================================")
    print(f"Training {model_name}...")
    clf.fit(train_embeddings_scaled, train_labels)

    # Evaluate on the Training Set
    print("Evaluation on Train Set:")
    train_macro_f1 = evaluate_model(clf, train_embeddings_scaled, train_labels, "Train Set", model_name)

    # Evaluate on the Testing Set
    print("Evaluation on Test Set:")
    test_macro_f1 = evaluate_model(clf, test_embeddings_scaled, test_labels, "Test Set", model_name)

    print(f"Summary for {model_name}:")
    print(f"  Train Macro F1 Score: {train_macro_f1}")
    print(f"  Test Macro F1 Score:  {test_macro_f1}")
    print("==============================================\n")

# -------------------------------
# 4. Predict Emotions for New Text
# -------------------------------

def predict_emotions(text, model, scaler):
    """
    Given a new text string, generate its embedding, scale it, and predict emotion labels.
    Returns a dictionary mapping each emotion to its predicted label (0 or 1).
    """
    # Generate embedding for the new text
    embedding = generate_embeddings([text])
    embedding_scaled = scaler.transform(embedding)

    # Get predictions
    predictions = model.predict(embedding_scaled)

    # Map each label to its corresponding emotion
    emotion_preds = {emotion: int(pred) for emotion, pred in zip(label_columns, predictions[0])}
    return emotion_preds

# Example usage for prediction using the SVM model (you can choose any of the trained models)
example_text = "यह एक उदाहरण वाक्य है"
print("Example prediction using SVM:")
svm_model = models["SVM"]
example_prediction = predict_emotions(example_text, svm_model, scaler)
for emotion, pred in example_prediction.items():
    print(f"{emotion}: {pred}")


Training SVM...
Evaluation on Train Set:

Model: SVM - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.97      0.91      0.94       492
     disgust       0.94      0.94      0.94       654
        fear       0.98      0.90      0.94       317
         joy       0.98      0.91      0.95       642
     sadness       1.00      0.87      0.93       309
    surprise       0.99      0.78      0.87       421

   micro avg       0.97      0.89      0.93      2835
   macro avg       0.98      0.88      0.93      2835
weighted avg       0.97      0.89      0.93      2835
 samples avg       0.95      0.91      0.92      2835

Macro F1 Score for Train Set: 0.9278488950697993

Evaluation on Test Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: SVM - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.64      0.68      0.66        34
     disgust       0.84      0.72      0.78        65
        fear       0.95      0.66      0.78        32
         joy       0.88      0.67      0.76        54
     sadness       0.93      0.42      0.58        31
    surprise       0.94      0.40      0.56        40

   micro avg       0.84      0.61      0.71       256
   macro avg       0.86      0.59      0.68       256
weighted avg       0.86      0.61      0.70       256
 samples avg       0.68      0.65      0.65       256

Macro F1 Score for Test Set: 0.6848093604074466

Summary for SVM:
  Train Macro F1 Score: 0.9278488950697993
  Test Macro F1 Score:  0.6848093604074466

Training Naive Bayes...
Evaluation on Train Set:

Model: Naive Bayes - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.66      0.83      0.73  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Evaluation on Train Set:

Model: Logistic Regression - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.96      0.95      0.96       492
     disgust       0.95      0.94      0.94       654
        fear       1.00      1.00      1.00       317
         joy       0.97      0.95      0.96       642
     sadness       1.00      0.99      1.00       309
    surprise       0.99      0.90      0.94       421

   micro avg       0.97      0.95      0.96      2835
   macro avg       0.98      0.95      0.97      2835
weighted avg       0.97      0.95      0.96      2835
 samples avg       0.97      0.96      0.96      2835

Macro F1 Score for Train Set: 0.9655484274823097

Evaluation on Test Set:

Model: Logistic Regression - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.47      0.65      0.54        34
     disgust       0.80      0.69      0.74        65
        fear    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Evaluation on Train Set:

Model: Random Forest - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.96      0.98       492
     disgust       1.00      0.99      0.99       654
        fear       1.00      0.94      0.97       317
         joy       1.00      0.97      0.99       642
     sadness       1.00      0.85      0.92       309
    surprise       1.00      0.91      0.95       421

   micro avg       1.00      0.95      0.97      2835
   macro avg       1.00      0.94      0.97      2835
weighted avg       1.00      0.95      0.97      2835
 samples avg       0.98      0.95      0.96      2835

Macro F1 Score for Train Set: 0.9662136544146662

Evaluation on Test Set:

Model: Random Forest - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.76      0.38      0.51        34
     disgust       0.88      0.45      0.59        65
        fear       0.94     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# ukranian

In [ ]:
import pandas as pd
import numpy as np

# Import classifiers and wrappers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, f1_score

# For generating text embeddings
from sentence_transformers import SentenceTransformer

# For scaling the embeddings
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load Data and Generate Embeddings
# -------------------------------

# Load datasets
train_df = pd.read_csv("/content/ukr.csv")
test_df = pd.read_csv("/content/ukr_dev.csv")

# Load the LaBSE model for generating embeddings
labse_model = SentenceTransformer('sentence-transformers/LaBSE')

def generate_embeddings(texts):
    """
    Given a list or array of texts, generate and return their normalized embeddings.
    """
    return np.array(labse_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# Get text data
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Generate embeddings for train and test sets
train_embeddings = generate_embeddings(train_texts)
test_embeddings = generate_embeddings(test_texts)

# -------------------------------
# 2. Prepare Labels and Scale Features
# -------------------------------

# Define the emotion label columns
label_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Extract label arrays
train_labels = train_df[label_columns].values
test_labels = test_df[label_columns].values

# Scale the embeddings (feature standardization)
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

# -------------------------------
# 3. Define, Train, and Evaluate Models
# -------------------------------

# Define the four models. For Random Forest we add some regularization
# by limiting the maximum tree depth and controlling the minimum samples needed for splits.
models = {
    "SVM": MultiOutputClassifier(SVC(kernel='rbf', probability=True, random_state=42)),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "Logistic Regression": MultiOutputClassifier(
        LogisticRegression(max_iter=1000, random_state=42)
    ),
    "Random Forest": MultiOutputClassifier(
        RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)
    )
}

def evaluate_model(model, embeddings, labels, dataset_name, model_name):
    """
    Predict labels using the given model and embeddings, then print the classification report
    and macro F1 score.
    """
    predictions = model.predict(embeddings)
    print(f"\nModel: {model_name} - {dataset_name}")
    print("Classification Report:")
    print(classification_report(labels, predictions, target_names=label_columns))

    macro_f1 = f1_score(labels, predictions, average='macro')
    print(f"Macro F1 Score for {dataset_name}: {macro_f1}\n")
    return macro_f1

# Train and evaluate each model on both the training and testing sets.
for model_name, clf in models.items():
    print("==============================================")
    print(f"Training {model_name}...")
    clf.fit(train_embeddings_scaled, train_labels)

    # Evaluate on the Training Set
    print("Evaluation on Train Set:")
    train_macro_f1 = evaluate_model(clf, train_embeddings_scaled, train_labels, "Train Set", model_name)

    # Evaluate on the Testing Set
    print("Evaluation on Test Set:")
    test_macro_f1 = evaluate_model(clf, test_embeddings_scaled, test_labels, "Test Set", model_name)

    print(f"Summary for {model_name}:")
    print(f"  Train Macro F1 Score: {train_macro_f1}")
    print(f"  Test Macro F1 Score:  {test_macro_f1}")
    print("==============================================\n")

# -------------------------------
# 4. Predict Emotions for New Text
# -------------------------------

def predict_emotions(text, model, scaler):
    """
    Given a new text string, generate its embedding, scale it, and predict emotion labels.
    Returns a dictionary mapping each emotion to its predicted label (0 or 1).
    """
    # Generate embedding for the new text
    embedding = generate_embeddings([text])
    embedding_scaled = scaler.transform(embedding)

    # Get predictions
    predictions = model.predict(embedding_scaled)

    # Map each label to its corresponding emotion
    emotion_preds = {emotion: int(pred) for emotion, pred in zip(label_columns, predictions[0])}
    return emotion_preds

# Example usage for prediction using the SVM model (you can choose any of the trained models)
example_text = "यह एक उदाहरण वाक्य है"
print("Example prediction using SVM:")
svm_model = models["SVM"]
example_prediction = predict_emotions(example_text, svm_model, scaler)
for emotion, pred in example_prediction.items():
    print(f"{emotion}: {pred}")


Training SVM...
Evaluation on Train Set:

Model: SVM - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.06      0.12        98
     disgust       1.00      0.07      0.13        86
        fear       0.99      0.84      0.91       172
         joy       0.98      0.78      0.87       412
     sadness       1.00      0.63      0.77       333
    surprise       1.00      0.55      0.71       196

   micro avg       0.99      0.61      0.76      1297
   macro avg       0.99      0.49      0.58      1297
weighted avg       0.99      0.61      0.72      1297
 samples avg       0.32      0.31      0.32      1297

Macro F1 Score for Train Set: 0.5839156789314446

Evaluation on Test Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: SVM - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        12
     disgust       0.00      0.00      0.00         9
        fear       0.90      0.53      0.67        17
         joy       0.60      0.29      0.39        41
     sadness       1.00      0.21      0.34        34
    surprise       1.00      0.11      0.19        19

   micro avg       0.77      0.23      0.35       132
   macro avg       0.58      0.19      0.27       132
weighted avg       0.70      0.23      0.32       132
 samples avg       0.12      0.12      0.12       132

Macro F1 Score for Test Set: 0.2653414824546372

Summary for SVM:
  Train Macro F1 Score: 0.5839156789314446
  Test Macro F1 Score:  0.2653414824546372

Training Naive Bayes...
Evaluation on Train Set:

Model: Naive Bayes - Train Set
Classification Report:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

              precision    recall  f1-score   support

       anger       0.28      0.77      0.41        98
     disgust       0.21      0.81      0.33        86
        fear       0.51      0.87      0.64       172
         joy       0.52      0.79      0.63       412
     sadness       0.41      0.78      0.54       333
    surprise       0.37      0.78      0.50       196

   micro avg       0.40      0.79      0.53      1297
   macro avg       0.38      0.80      0.51      1297
weighted avg       0.43      0.79      0.55      1297
 samples avg       0.31      0.40      0.33      1297

Macro F1 Score for Train Set: 0.5090833556781851

Evaluation on Test Set:

Model: Naive Bayes - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.25      0.50      0.33        12
     disgust       0.15      0.44      0.22         9
        fear       0.52      0.94      0.67        17
         joy       0.45      0.71      0.55        41
    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Random Forest - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.04      0.08        98
     disgust       1.00      0.05      0.09        86
        fear       1.00      0.72      0.84       172
         joy       1.00      0.87      0.93       412
     sadness       1.00      0.56      0.72       333
    surprise       1.00      0.59      0.74       196

   micro avg       1.00      0.61      0.76      1297
   macro avg       1.00      0.47      0.57      1297
weighted avg       1.00      0.61      0.72      1297
 samples avg       0.32      0.31      0.31      1297

Macro F1 Score for Train Set: 0.5671798982355136

Evaluation on Test Set:

Model: Random Forest - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        12
     disgust       0.00      0.00      0.00         9
        fear       0.00     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

# english

In [ ]:
import pandas as pd
import numpy as np

# Import classifiers and wrappers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, f1_score

# For generating text embeddings
from sentence_transformers import SentenceTransformer

# For scaling the embeddings
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load Data and Generate Embeddings
# -------------------------------

# Load datasets
train_df = pd.read_csv("/content/esp.csv")
test_df = pd.read_csv("/content/esp_dev.csv")

# Load the LaBSE model for generating embeddings
labse_model = SentenceTransformer('sentence-transformers/LaBSE')

def generate_embeddings(texts):
    """
    Given a list or array of texts, generate and return their normalized embeddings.
    """
    return np.array(labse_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# Get text data
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Generate embeddings for train and test sets
train_embeddings = generate_embeddings(train_texts)
test_embeddings = generate_embeddings(test_texts)

# -------------------------------
# 2. Prepare Labels and Scale Features
# -------------------------------

# Define the expected emotion labels.
expected_labels = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Create label_columns by including only those labels that exist in both datasets.
label_columns = [label for label in expected_labels if label in train_df.columns and label in test_df.columns]

# Optionally, print which labels will be used.
print("Using emotion labels:", label_columns)

# Extract label arrays
train_labels = train_df[label_columns].values
test_labels = test_df[label_columns].values

# Scale the embeddings (feature standardization)
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

# -------------------------------
# 3. Define, Train, and Evaluate Models
# -------------------------------

# Define the four models. For Random Forest we add some regularization
# by limiting the maximum tree depth and controlling the minimum samples needed for splits.
models = {
    "SVM": MultiOutputClassifier(SVC(kernel='rbf', probability=True, random_state=42)),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "Logistic Regression": MultiOutputClassifier(
        LogisticRegression(max_iter=1000, random_state=42)
    ),
    "Random Forest": MultiOutputClassifier(
        RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)
    )
}

def evaluate_model(model, embeddings, labels, dataset_name, model_name):
    """
    Predict labels using the given model and embeddings, then print the classification report
    and macro F1 score.
    """
    predictions = model.predict(embeddings)
    print(f"\nModel: {model_name} - {dataset_name}")
    print("Classification Report:")
    print(classification_report(labels, predictions, target_names=label_columns))

    macro_f1 = f1_score(labels, predictions, average='macro')
    print(f"Macro F1 Score for {dataset_name}: {macro_f1}\n")
    return macro_f1

# Train and evaluate each model on both the training and testing sets.
for model_name, clf in models.items():
    print("==============================================")
    print(f"Training {model_name}...")
    clf.fit(train_embeddings_scaled, train_labels)

    # Evaluate on the Training Set
    print("Evaluation on Train Set:")
    train_macro_f1 = evaluate_model(clf, train_embeddings_scaled, train_labels, "Train Set", model_name)

    # Evaluate on the Testing Set
    print("Evaluation on Test Set:")
    test_macro_f1 = evaluate_model(clf, test_embeddings_scaled, test_labels, "Test Set", model_name)

    print(f"Summary for {model_name}:")
    print(f"  Train Macro F1 Score: {train_macro_f1}")
    print(f"  Test Macro F1 Score:  {test_macro_f1}")
    print("==============================================\n")

# -------------------------------
# 4. Predict Emotions for New Text
# -------------------------------

def predict_emotions(text, model, scaler):
    """
    Given a new text string, generate its embedding, scale it, and predict emotion labels.
    Returns a dictionary mapping each available emotion to its predicted label (0 or 1).
    """
    # Generate embedding for the new text
    embedding = generate_embeddings([text])
    embedding_scaled = scaler.transform(embedding)

    # Get predictions
    predictions = model.predict(embedding_scaled)

    # Map each label to its corresponding emotion
    emotion_preds = {emotion: int(pred) for emotion, pred in zip(label_columns, predictions[0])}
    return emotion_preds

# Example usage for prediction using the SVM model (you can choose any of the trained models)
example_text = "यह एक उदाहरण वाक्य है"
print("Example prediction using SVM:")
svm_model = models["SVM"]
example_prediction = predict_emotions(example_text, svm_model, scaler)
for emotion, pred in example_prediction.items():
    print(f"{emotion}: {pred}")


Using emotion labels: ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']
Training SVM...
Evaluation on Train Set:

Model: SVM - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.97      0.91      0.94       492
     disgust       0.94      0.94      0.94       654
        fear       0.98      0.90      0.94       317
         joy       0.98      0.91      0.95       642
     sadness       1.00      0.87      0.93       309
    surprise       0.99      0.78      0.87       421

   micro avg       0.97      0.89      0.93      2835
   macro avg       0.98      0.88      0.93      2835
weighted avg       0.97      0.89      0.93      2835
 samples avg       0.95      0.91      0.92      2835

Macro F1 Score for Train Set: 0.9278488950697993

Evaluation on Test Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: SVM - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.64      0.68      0.66        34
     disgust       0.84      0.72      0.78        65
        fear       0.95      0.66      0.78        32
         joy       0.88      0.67      0.76        54
     sadness       0.93      0.42      0.58        31
    surprise       0.94      0.40      0.56        40

   micro avg       0.84      0.61      0.71       256
   macro avg       0.86      0.59      0.68       256
weighted avg       0.86      0.61      0.70       256
 samples avg       0.68      0.65      0.65       256

Macro F1 Score for Test Set: 0.6848093604074466

Summary for SVM:
  Train Macro F1 Score: 0.9278488950697993
  Test Macro F1 Score:  0.6848093604074466

Training Naive Bayes...
Evaluation on Train Set:

Model: Naive Bayes - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.66      0.83      0.73  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Evaluation on Train Set:

Model: Logistic Regression - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.96      0.95      0.96       492
     disgust       0.95      0.94      0.94       654
        fear       1.00      1.00      1.00       317
         joy       0.97      0.95      0.96       642
     sadness       1.00      0.99      1.00       309
    surprise       0.99      0.90      0.94       421

   micro avg       0.97      0.95      0.96      2835
   macro avg       0.98      0.95      0.97      2835
weighted avg       0.97      0.95      0.96      2835
 samples avg       0.97      0.96      0.96      2835

Macro F1 Score for Train Set: 0.9655484274823097

Evaluation on Test Set:

Model: Logistic Regression - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.47      0.65      0.54        34
     disgust       0.80      0.69      0.74        65
        fear    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Evaluation on Train Set:

Model: Random Forest - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.96      0.98       492
     disgust       1.00      0.99      0.99       654
        fear       1.00      0.94      0.97       317
         joy       1.00      0.97      0.99       642
     sadness       1.00      0.85      0.92       309
    surprise       1.00      0.91      0.95       421

   micro avg       1.00      0.95      0.97      2835
   macro avg       1.00      0.94      0.97      2835
weighted avg       1.00      0.95      0.97      2835
 samples avg       0.98      0.95      0.96      2835

Macro F1 Score for Train Set: 0.9662136544146662

Evaluation on Test Set:

Model: Random Forest - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.76      0.38      0.51        34
     disgust       0.88      0.45      0.59        65
        fear       0.94     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
